In [1]:
import numpy as np
import pandas as pd
import RNA # vienna RNA
import random
import matplotlib.pyplot as plt
import os

In [2]:
path_data='./data/ss_largeSizedDatasetRFAMonly.db'
path_data_output='./data/ss_largeSizedDatasetRFAMonly/'

path_data_output_clean_indiv=path_data_output+'individual/'
import os
for i in os.listdir(path_data_output_clean_indiv):
    os.remove(path_data_output_clean_indiv+i)


In [3]:
#Primary
output_path_structureValidated_train_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structure_train_data.csv'
output_path_structureMFE_train_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structureMFE_train_data.csv'

output_path_structureValidated_valid_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structure_valid_data.csv'
output_path_structureMFE_valid_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structureMFE_valid_data.csv'

output_path_structureValidated_test_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structure_test_data.csv'
output_path_structureMFE_test_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structureMFE_test_data.csv'

#Secondary
output_path_structureValidAndMfe_db=path_data_output+'ss_largeSizedDatasetRFAMonly_structureValidAndMFE_data.db'

Read db data

In [4]:
f=open(path_data)
rna_id=[]
rna_seq=[]
rna_ss=[]
count=0
for line in f:
    line=line.strip()
    if line.startswith('>'):
        rna_id.append(line.strip('>').replace(';','__'))
        temp='rna_id' 
        count+=1
    elif count==1:
        rna_seq.append(line)
        count+=1   
    elif count==2:
        rna_ss.append(line)
        count=0   

In [5]:
print(rna_id[1])
print(rna_seq[1])
print(rna_ss[1])
print('Number of RNA : '+str(len(rna_id)) )

2_U1_RF00003_X14417
AUACUUACCUGGACGGGGUCAAUGGGUAAUCAAGAAGUUCCAUGGCCUAGGUUGGUGACCUCCAUUGCACUAAGGAGGGGUGCUUGCCUAAGGUCGACCCAAGUGGUUGAGCCUACGUCAUAAUUUGUUGUUGCAGAGGGGGCCUGUGUUCGCGCAGCCCCUAC
...........((((((((((.((((............))))))))))((((.(((..(((((..........))).))..))).)))).(((((((((.....)))))).))).))))....(((((....)))))(((((.(((((....))))))))))..
Number of RNA : 12512


In [6]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__   

Generate subopt for each sequences and output db for each and one db for all

In [7]:
from tqdm import tqdm


with open(output_path_structureValidAndMfe_db, 'w') as the_file:

    for i in tqdm(range(len(rna_id))):
        
        fc = RNA.fold_compound(rna_seq[i])
        (mfe_struct, mfe) = fc.mfe()

        the_file.write('>'+rna_id[i]+'__'+'0_0'+'__is_subopt_0'+'__ss_0'+'\n')
        the_file.write(rna_seq[i]+'\n')
        the_file.write(rna_ss[i]+'\n')

        ss_subopts_bp_score=RNA.bp_distance(mfe_struct,rna_ss[i])/(mfe_struct.count('(')+rna_ss[i].count('('))
        the_file.write('>'+rna_id[i]+'__'+str(round(ss_subopts_bp_score,2)).replace('.','_')+'__is_subopt_1'+'__ss_0'+'\n')
        the_file.write(rna_seq[i]+'\n')
        the_file.write(mfe_struct+'\n')


100%|████████████████████████████████████| 12512/12512 [01:14<00:00, 168.21it/s]


In [4]:
# Get all sequence content
f=open(output_path_structureValidAndMfe_db)
rna_id_sequences=[]
rna_sequences=[]
count=0
for line in f:
    line=line.strip()
    if line.startswith('>'):
        rna_id_sequences.append(line.strip('>'))
        temp='rna_id_sequences' 
        count+=1
    elif count==1:
        rna_sequences.append(line)
        count=0
        

In [5]:
print('Number of RNA : '+str(len(rna_sequences)))
print(rna_id_sequences[0])

Number of RNA : 25024
1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0


Compute n-motifs rep

In [6]:
import os

os.system('./nmotifs'+' -i ' + output_path_structureValidAndMfe_db + ' -o '+ path_data_output+' -m 2')
#os.system('./nmotifs'+' -i ' + output_path_structureValidAndMfe_db + ' -o '+ path_data_output+' -m 10')
#os.system('./nmotifs'+' -i ' + output_path_structureValidAndMfe_db + ' -o '+ path_data_output)
#os.system('./nmotifs'+' -i ' + output_path_structureValidAndMfe_db + ' -o '+ path_data_output +' -n 0')



Running the super n-motifs program...

Extract n-motifs...
Took:1.87511 sec.
Filter and weight n-motifs to build the n-motifs representation ...
Took:15.1207 sec.

Compute and Write output matNmRep_SSbyNm.csv & matnmPos.csv files:
SS*n-motifs matrix that is the n-motif representation of SS (matNmRep_SSbyNm.csv) ...
n-motifsPosition matrix (matnmPos.csv) ...
n-motifsSequence matrix (matnmSeq.csv) ...
Execution of n-motifs program completed
Took:245.865 sec.


0

In [7]:
nm_rep_df=pd.read_csv(path_data_output+'matNmRep_SSbyNm.csv',index_col=[0])

In [8]:
name_nm_rep_df=nm_rep_df.index.values
name_nm_rep_df

array(['1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0',
       '1_U1_RF00003_X06810__0_18__is_subopt_1__ss_0',
       '2_U1_RF00003_X14417__0_0__is_subopt_0__ss_0', ...,
       '116_sau-50_RF02391_ADMF01000015__1_0__is_subopt_1__ss_0',
       '117_sau-50_RF02391_CP002643__0_0__is_subopt_0__ss_0',
       '117_sau-50_RF02391_CP002643__0_19__is_subopt_1__ss_0'],
      dtype=object)

In [9]:
#nm_rep_df=np.log2(nm_rep_df+1)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# nm_rep_df=pd.DataFrame(scaler.fit_transform(nm_rep_df))
# nm_rep_df.index=name_nm_rep_df

nm_rep_df

,B,B[E3E5SS],B[E3SS],B[E5SS],B[SSS],B_1,B_10,B_10[S_10S_3],B_10[S_11S_7],B_10[S_2S_4],...,S_9[M_10M_6],S_9[M_11M_5],S_9[M_13M_15],S_9[M_17M_18],S_9[M_3M_8],S_9[M_5M_5],S_9[M_5M_6],S_9[M_5M_8],S_9[M_6M_6],S_9[M_6M_8]
,,,,,,,,,,,,,,,,,,,,,
1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1_U1_RF00003_X06810__0_18__is_subopt_1__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_0__is_subopt_0__ss_0,4,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_06__is_subopt_1__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3_U1_RF00003_Z11883__0_0__is_subopt_0__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115_sau-50_RF02391_CP002657__1_0__is_subopt_1__ss_0,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__0_0__is_subopt_0__ss_0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__1_0__is_subopt_1__ss_0,3,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(analyzer='char', ngram_range=(5, 5))
# kmer_ = vectorizer.fit_transform(rna_sequences)
# nm_seq_rep_df=pd.DataFrame(kmer_.toarray(),columns=list(vectorizer.get_feature_names_out()), index=name_nm_rep_df)
# print(nm_seq_rep_df.shape)

#nm_seq_rep_df=np.log2(nm_seq_rep_df+1)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# nm_seq_rep_df=pd.DataFrame(scaler.fit_transform(nm_seq_rep_df))
# nm_seq_rep_df.index=name_nm_rep_df

#nm_seq_rep_df

NameError: name 'nm_seq_rep_df' is not defined

In [16]:
#nm_rep_df=pd.concat([nm_rep_df, nm_seq_rep_df,matnmSeq_nm], axis=1)
#nm_rep_df=pd.concat([nm_rep_df, nm_seq_rep_df], axis=1)

nm_rep_df

,B,B[E3E5SS],B[E3SS],B[E5SS],B[SSS],B_1,B_10,B_10[S_10S_3],B_10[S_11S_7],B_10[S_2S_4],...,S_9[M_10M_6],S_9[M_11M_5],S_9[M_13M_15],S_9[M_17M_18],S_9[M_3M_8],S_9[M_5M_5],S_9[M_5M_6],S_9[M_5M_8],S_9[M_6M_6],S_9[M_6M_8]
,,,,,,,,,,,,,,,,,,,,,
1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1_U1_RF00003_X06810__0_18__is_subopt_1__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_0__is_subopt_0__ss_0,4,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_06__is_subopt_1__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3_U1_RF00003_Z11883__0_0__is_subopt_0__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115_sau-50_RF02391_CP002657__1_0__is_subopt_1__ss_0,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__0_0__is_subopt_0__ss_0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__1_0__is_subopt_1__ss_0,3,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
nm_rep_df

,B,B[E3E5SS],B[E3SS],B[E5SS],B[SSS],B_1,B_10,B_10[S_10S_3],B_10[S_11S_7],B_10[S_2S_4],...,S_9[M_10M_6],S_9[M_11M_5],S_9[M_13M_15],S_9[M_17M_18],S_9[M_3M_8],S_9[M_5M_5],S_9[M_5M_6],S_9[M_5M_8],S_9[M_6M_6],S_9[M_6M_8]
,,,,,,,,,,,,,,,,,,,,,
1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1_U1_RF00003_X06810__0_18__is_subopt_1__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_0__is_subopt_0__ss_0,4,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_06__is_subopt_1__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3_U1_RF00003_Z11883__0_0__is_subopt_0__ss_0,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115_sau-50_RF02391_CP002657__1_0__is_subopt_1__ss_0,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__0_0__is_subopt_0__ss_0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__1_0__is_subopt_1__ss_0,3,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
name_nm_rep_df=nm_rep_df.index.values
family_nm_rep_df=[i.split('_')[1] for i in name_nm_rep_df]
is_subopt=[i.split('__')[2] for i in name_nm_rep_df]
bd_distance=[float(i.split('__')[1].replace('_','.')) for i in name_nm_rep_df]

nm_rep_df.insert(0, "family", family_nm_rep_df, True)
nm_rep_df.insert(0, "id_seq", name_nm_rep_df, True)
nm_rep_df.insert(0, "is_subopt", is_subopt, True)
nm_rep_df.insert(0, "bd_distance", bd_distance, True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
family_int=le.fit_transform(nm_rep_df['family'])
nm_rep_df.insert(0, "family_int", family_int, True)

nm_rep_df

,bd_distance,is_subopt,id_seq,family,B,B[E3E5SS],B[E3SS],B[E5SS],B[SSS],B_1,...,S_9[M_10M_6],S_9[M_11M_5],S_9[M_13M_15],S_9[M_17M_18],S_9[M_3M_8],S_9[M_5M_5],S_9[M_5M_6],S_9[M_5M_8],S_9[M_6M_6],S_9[M_6M_8]
,,,,,,,,,,,,,,,,,,,,,
1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0,0.00,is_subopt_0,1_U1_RF00003_X06810__0_0__is_subopt_0__ss_0,U1,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1_U1_RF00003_X06810__0_18__is_subopt_1__ss_0,0.18,is_subopt_1,1_U1_RF00003_X06810__0_18__is_subopt_1__ss_0,U1,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_0__is_subopt_0__ss_0,0.00,is_subopt_0,2_U1_RF00003_X14417__0_0__is_subopt_0__ss_0,U1,4,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2_U1_RF00003_X14417__0_06__is_subopt_1__ss_0,0.06,is_subopt_1,2_U1_RF00003_X14417__0_06__is_subopt_1__ss_0,U1,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3_U1_RF00003_Z11883__0_0__is_subopt_0__ss_0,0.00,is_subopt_0,3_U1_RF00003_Z11883__0_0__is_subopt_0__ss_0,U1,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115_sau-50_RF02391_CP002657__1_0__is_subopt_1__ss_0,1.00,is_subopt_1,115_sau-50_RF02391_CP002657__1_0__is_subopt_1_...,sau-50,4,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__0_0__is_subopt_0__ss_0,0.00,is_subopt_0,116_sau-50_RF02391_ADMF01000015__0_0__is_subop...,sau-50,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
116_sau-50_RF02391_ADMF01000015__1_0__is_subopt_1__ss_0,1.00,is_subopt_1,116_sau-50_RF02391_ADMF01000015__1_0__is_subop...,sau-50,3,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


# Select Family Test

In [21]:
from collections import Counter
print (Counter(family_nm_rep_df))

{'msiK', 'sau-50', 'TwoAYGGAY', 'JUMPstart', 'U4', 'Flavivirus', 'ar14', 'MOCO', 'U1', 'Purine', 'TPP', 'mir-1937', 'group-II-D1D4-1', 'ykoK', 'crcB', 'PK-G12rRNA', 'group-II-D1D4-2', 'U2', 'PhotoRC-II', 'AdoCbl-variant', 'SAM-II', 'Downstream-peptide', 'c-di-GMP-I', 'potC', 'group-II-D1D4-5', 'ydaO-yuaA', 'IsrR', 'radC', 'group-II-D1D4-7', 'pfl', 'HEARO', 'FMN', 'HCV', 'group-II-D1D4-3', '6S', 'MIR169', 'wcaG', 'SAM-I-IV-variant', 'Y', 'IMES-1', 'Cobalamin', 'glnA', 'Cyano-1', 'manA', 'group-II-D1D4-6', 'HIV', 'Entero', 'Prion', 'U6', 'C4', 'mir-154', 'Flavo-1', 'SAM', 'IMES-2', 'U5', 'Clostridiales-1'}
Counter({'glnA': 1912, 'Downstream-peptide': 1254, 'C4': 912, 'PhotoRC-II': 890, 'SAM-I-IV-variant': 874, 'SAM': 866, 'Cobalamin': 860, 'group-II-D1D4-3': 812, 'IMES-2': 732, 'PK-G12rRNA': 694, 'IsrR': 616, 'crcB': 574, 'HIV': 550, 'Cyano-1': 530, 'group-II-D1D4-7': 488, 'group-II-D1D4-1': 474, 'HCV': 440, 'IMES-1': 438, 'TwoAYGGAY': 420, 'U2': 416, 'Flavo-1': 402, 'Clostridiales-1': 3

In [22]:
family_test='Cobalamin'
family_valid='6S'

# Prepare Data for learning, validation and testing

# Training Data

In [23]:
print('**********************')
print ('Write training data : validated structure')
nm_rep_df.loc[(nm_rep_df['family']!=family_test) & (nm_rep_df['family']!=family_valid)& (nm_rep_df['is_subopt']!='is_subopt_0')].to_csv(output_path_structureValidated_train_data)
print ('Write training data : mfe structure')
nm_rep_df.loc[(nm_rep_df['family']!=family_test) & (nm_rep_df['family']!=family_valid)& (nm_rep_df['is_subopt']!='is_subopt_1')].to_csv(output_path_structureMFE_train_data)
print('**********************')

**********************
Write training data : validated structure
Write training data : mfe structure
**********************


# Validation Data

In [26]:
validation_data=nm_rep_df[nm_rep_df['family']==family_valid]
print (Counter(validation_data['family']))

validation_data_validated_structure=validation_data[validation_data['is_subopt'] == 'is_subopt_0']
validation_data_mfe_structure=validation_data[validation_data['is_subopt'] == 'is_subopt_1']

validation_data_validated_structure.to_csv(output_path_structureValidated_valid_data)
validation_data_mfe_structure.to_csv(output_path_structureMFE_valid_data)

Counter({'6S': 298})


In [27]:
validation_data_mfe_structure

,bd_distance,is_subopt,id_seq,family,B,B[E3E5SS],B[E3SS],B[E5SS],B[SSS],B_1,...,S_9[M_10M_6],S_9[M_11M_5],S_9[M_13M_15],S_9[M_17M_18],S_9[M_3M_8],S_9[M_5M_5],S_9[M_5M_6],S_9[M_5M_8],S_9[M_6M_6],S_9[M_6M_8]
,,,,,,,,,,,,,,,,,,,,,
1_6S_RF00013_CR543861__0_07__is_subopt_1__ss_0,0.07,is_subopt_1,1_6S_RF00013_CR543861__0_07__is_subopt_1__ss_0,6S,4,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2_6S_RF00013_AB036820__0_41__is_subopt_1__ss_0,0.41,is_subopt_1,2_6S_RF00013_AB036820__0_41__is_subopt_1__ss_0,6S,4,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3_6S_RF00013_AL591979__0_14__is_subopt_1__ss_0,0.14,is_subopt_1,3_6S_RF00013_AL591979__0_14__is_subopt_1__ss_0,6S,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4_6S_RF00013_BX842646__0_01__is_subopt_1__ss_0,0.01,is_subopt_1,4_6S_RF00013_BX842646__0_01__is_subopt_1__ss_0,6S,4,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5_6S_RF00013_AE017355__0_2__is_subopt_1__ss_0,0.20,is_subopt_1,5_6S_RF00013_AE017355__0_2__is_subopt_1__ss_0,6S,4,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145_6S_RF00013_CP002469__0_02__is_subopt_1__ss_0,0.02,is_subopt_1,145_6S_RF00013_CP002469__0_02__is_subopt_1__ss_0,6S,5,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
146_6S_RF00013_CP002222__0_07__is_subopt_1__ss_0,0.07,is_subopt_1,146_6S_RF00013_CP002222__0_07__is_subopt_1__ss_0,6S,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
147_6S_RF00013_CP003056__0_22__is_subopt_1__ss_0,0.22,is_subopt_1,147_6S_RF00013_CP003056__0_22__is_subopt_1__ss_0,6S,6,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


# Test Data

In [28]:
test_data=nm_rep_df[nm_rep_df['family']==family_test]
print (Counter(test_data['family']))

test_data_validated_structure=test_data[test_data['is_subopt'] == 'is_subopt_0']
test_data_mfe_structure=test_data[test_data['is_subopt'] == 'is_subopt_1']

test_data_validated_structure.to_csv(output_path_structureValidated_test_data)
test_data_mfe_structure.to_csv(output_path_structureMFE_test_data)

Counter({'Cobalamin': 860})


In [29]:
test_data_mfe_structure

,bd_distance,is_subopt,id_seq,family,B,B[E3E5SS],B[E3SS],B[E5SS],B[SSS],B_1,...,S_9[M_10M_6],S_9[M_11M_5],S_9[M_13M_15],S_9[M_17M_18],S_9[M_3M_8],S_9[M_5M_5],S_9[M_5M_6],S_9[M_5M_8],S_9[M_6M_6],S_9[M_6M_8]
,,,,,,,,,,,,,,,,,,,,,
1_Cobalamin_RF00174_AF010496__0_54__is_subopt_1__ss_0,0.54,is_subopt_1,1_Cobalamin_RF00174_AF010496__0_54__is_subopt_...,Cobalamin,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2_Cobalamin_RF00174_AF010496__0_45__is_subopt_1__ss_0,0.45,is_subopt_1,2_Cobalamin_RF00174_AF010496__0_45__is_subopt_...,Cobalamin,5,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3_Cobalamin_RF00174_AF010496__0_35__is_subopt_1__ss_0,0.35,is_subopt_1,3_Cobalamin_RF00174_AF010496__0_35__is_subopt_...,Cobalamin,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4_Cobalamin_RF00174_AF193754__0_6__is_subopt_1__ss_0,0.60,is_subopt_1,4_Cobalamin_RF00174_AF193754__0_6__is_subopt_1...,Cobalamin,3,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5_Cobalamin_RF00174_AF193754__0_19__is_subopt_1__ss_0,0.19,is_subopt_1,5_Cobalamin_RF00174_AF193754__0_19__is_subopt_...,Cobalamin,4,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426_Cobalamin_RF00174_CP002156__0_79__is_subopt_1__ss_0,0.79,is_subopt_1,426_Cobalamin_RF00174_CP002156__0_79__is_subop...,Cobalamin,4,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
427_Cobalamin_RF00174_AALC02000021__0_43__is_subopt_1__ss_0,0.43,is_subopt_1,427_Cobalamin_RF00174_AALC02000021__0_43__is_s...,Cobalamin,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
428_Cobalamin_RF00174_CP003056__0_33__is_subopt_1__ss_0,0.33,is_subopt_1,428_Cobalamin_RF00174_CP003056__0_33__is_subop...,Cobalamin,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
